In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ita-footballcorpusv2/football_corpus_chunks_A_B.csv
/kaggle/input/ita-footballcorpus/football_corpus_chunksize_240.csv


# **Intro to text Analytics assignment 4**

Shahmeer Khan
25156

Muhammad Ibrahim Farid
27098

Intructor: Miss Solat Jabeen Sheikh


In [2]:
!pip install -U langchain-community
!pip install nltk
!pip install rank-bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 25.4 MB/s eta 0:00:00
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.3
    Uninstalling langchain-text-splitters-0.3.3:
      Successfully uninstalled langchain-text-splitters-0.3.3
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.

Installing relevant packages

In [3]:
import nltk
import subprocess

# Download and unzip wordnet
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

# Now you can import the NLTK resources
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

[nltk_data] Downloading package wordnet to /kaggle/working/...


Process to import nltk resources, simple import doesnt work for Kaggle

In [4]:
import pandas as pd
import numpy as np
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
from sklearn.preprocessing import normalize
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline
import warnings
warnings.filterwarnings("ignore")
import textwrap
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
from huggingface_hub import login
import torch
from collections import defaultdict


Importing relevant libraries

In [5]:
# Loading the corpus, data preparation explained in report
file_path = '/kaggle/input/ita-footballcorpusv2/football_corpus_chunks_A_B.csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1')

print(df.head())

     chunk                                            content
0  chunk 1  Association football, more commonly known as f...
1  chunk 2  that has scored more goals at the end of the g...
2  chunk 3  such as Australia, Canada, South Africa, most ...
3  chunk 4  having to avoid it touching the ground at any ...
4  chunk 5  football, wrestling, and volleyball more than ...


Loading the corpus and displaying its first 5 rows including column names etc

In [6]:
# Example frequent football bigrams, use to differentiate between gaol, kick, and goalkick etc
frequent_bigrams = [
    "goal kick", "corner kick", "free kick", "penalty kick",
    "offside trap", "yellow card", "red card", "extra time",
    "injury time", "penalty shootout", "kick off", "full time",
    "half time"
]

# Preprocess function
def replace_bigrams(text, bigrams):
    for bigram in bigrams:
        joined = bigram.replace(" ", "_")
        # Replace with word boundaries to avoid mid-word matches
        text = re.sub(r'\b' + re.escape(bigram) + r'\b', joined, text, flags=re.IGNORECASE)
    return text

# Applying preprocessing
df['content'] = df['content'].astype(str).apply(lambda x: replace_bigrams(x, frequent_bigrams))

# Convert to list of documents
documents = df['content'].tolist()

Some terms in the corpus like goal kick and corner kick frequently appeared as bigrams together so concatenated them to use them as unigrams instead to make tokenization easier and thus cosine similarity etc

In [7]:
# Defining and using embedding techniques
def tfidf_embedding(documents):
    tfidf = TfidfVectorizer(max_features=5000)  # Limit to 5000 features for better performance
    embeddings = tfidf.fit_transform(documents).toarray()
    return embeddings

def svd_embedding(documents, n_components=100):
    tfidf = TfidfVectorizer(max_features=5000)
    tfidf_matrix = tfidf.fit_transform(documents)
    svd = TruncatedSVD(n_components=n_components)
    embeddings = svd.fit_transform(tfidf_matrix)
    return embeddings

def sentence_transformers_embedding(documents):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(documents)
    return embeddings

def doc2vec_embedding(documents, vector_size=100, window=5, min_count=1, workers=4):
    tagged_data = [TaggedDocument(words=doc.split(), tags=[str(i)]) for i, doc in enumerate(documents)]
    model = Doc2Vec(vector_size=vector_size, window=window, min_count=min_count, workers=workers)
    model.build_vocab(tagged_data)
    model.train(tagged_data, total_examples=model.corpus_count, epochs=10)
    embeddings = [model.infer_vector(doc.split()) for doc in documents]
    return embeddings

# Function to select embedding technique
def select_embedding_technique(documents, technique='tfidf'):
    if technique == 'tfidf':
        return tfidf_embedding(documents)
    elif technique == 'svd':
        return svd_embedding(documents)
    elif technique == 'sentence_transformers':
        return sentence_transformers_embedding(documents)
    elif technique == 'doc2vec':
        return doc2vec_embedding(documents)
    else:
        raise ValueError("Invalid technique. Choose from ['tfidf', 'svd', 'sentence_transformers', 'doc2vec']")

A selection function to choose between 4 embedding techniques TF-IDF,SVD,sentence transformer and Doc2Vec, this one is for document embeddings

In [8]:
#Similar function as before, this one generates embeddings for queries
def generate_query_embedding(query, technique='tfidf'):
    if technique == 'tfidf':
        # Using TfidfVectorizer
        tfidf = TfidfVectorizer(max_features=5000)
        tfidf_matrix = tfidf.fit_transform(documents)  
        query_embedding = tfidf.transform([query]).toarray() 
        return query_embedding

    elif technique == 'svd':

        tfidf = TfidfVectorizer(max_features=5000)
        tfidf_matrix = tfidf.fit_transform(documents) 
        svd = TruncatedSVD(n_components=100)
        svd_matrix = svd.fit_transform(tfidf_matrix)
        query_embedding = svd.transform(tfidf.transform([query]).toarray())
        return query_embedding

    elif technique == 'sentence_transformers':

        model = SentenceTransformer('all-MiniLM-L6-v2') 
        query_embedding = model.encode([query])
        return query_embedding

    elif technique == 'doc2vec':

        model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4)
        tagged_data = [TaggedDocument(words=doc.split(), tags=[str(i)]) for i, doc in enumerate(documents)]
        model.build_vocab(tagged_data)
        model.train(tagged_data, total_examples=model.corpus_count, epochs=10)
        query_embedding = model.infer_vector(query.split())
        return query_embedding

    else:
        raise ValueError("Invalid technique. Choose from ['tfidf', 'svd', 'sentence_transformers', 'doc2vec']")


Similar selection function as before except to be used for query embedding, make sure its the same for document embedding

In [9]:
# Initialize stopwords and lemmatizer, could be later used for testing with and without this
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Tokenize the text into words
    words = word_tokenize(text.lower())  # Convert to lowercase and tokenize

    # Remove stopwords and apply lemmatization
    processed_words = [lemmatizer.lemmatize(word) for word in words if word.isalpha() and word not in stop_words]

    return processed_words

tokenized_documents = [preprocess_text(doc) for doc in documents]
bm25 = BM25Okapi(tokenized_documents)
    
# Function to retrieve top-k relevant documents, allows choice between semantic (cosine), bm25(keyword), hybrid
def retrieve_top_k_documents(query, documents, document_embeddings, k=10, search_type='cosine',fusion_k=60):
    if search_type == 'cosine':
        # Generate the query embedding using the same embedding model
        query_embedding = generate_query_embedding(query, technique='svd')
        query_embedding = query_embedding.reshape(1, -1)

        # Compute cosine similarity
        cosine_sim = cosine_similarity(query_embedding, document_embeddings)
        top_k_indices = cosine_sim[0].argsort()[-k:][::-1]
        return top_k_indices, cosine_sim[0][top_k_indices]

    elif search_type == 'bm25':
        # Preprocess the query
        tokenized_query = preprocess_text(query)

        # Retrieve BM25 scores
        scores = bm25.get_scores(tokenized_query)
        top_k_indices = np.argsort(scores)[-k:][::-1]
        top_k_scores = scores[top_k_indices]
        return top_k_indices, top_k_scores

    elif search_type == 'hybrid':
        # BM25 results
        tokenized_query = preprocess_text(query)
        bm25_scores = bm25.get_scores(tokenized_query)
        top_bm25_indices = np.argsort(bm25_scores)[-fusion_k:][::-1]
        results_bm25 = [(documents[i], bm25_scores[i]) for i in top_bm25_indices]

        # Cosine results
        query_embedding = generate_query_embedding(query, technique='svd').reshape(1, -1)
        cosine_sim = cosine_similarity(query_embedding, document_embeddings)[0]
        top_cosine_indices = cosine_sim.argsort()[-fusion_k:][::-1]
        results_cosine = [(documents[i], cosine_sim[i]) for i in top_cosine_indices]

        # RRF Fusion )for reranking documents in hybrid approach
        fused_scores = {}
        for rank, (doc, _) in enumerate(results_bm25):
            fused_scores[doc] = fused_scores.get(doc, 0) + 1 / (rank + 1)
        for rank, (doc, _) in enumerate(results_cosine):
            fused_scores[doc] = fused_scores.get(doc, 0) + 1 / (rank + 1)

        # Sort final scores and return top-k documents and scores
        sorted_fused = sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)[:k]
        top_k_docs = [doc for doc, score in sorted_fused]
        top_k_scores = [score for doc, score in sorted_fused]

        top_k_indices = [documents.index(doc) for doc in top_k_docs]
        return top_k_indices, top_k_scores

    else:
        raise ValueError("Invalid search_type. Choose from ['cosine', 'bm25', 'hybrid']")

Implementing preprocessing (lemmatization and stopword removal) for BM25 and hybrid implementation, a document retrieval function which also allows selection between cosine similarity technique, BM25 (keyword extraction) and a hybrid approach which uses RRF for document ranking

In [10]:
document_embeddings = select_embedding_technique(documents, technique='svd')

Generating document embeddings for our corpus using the embedding function, svd by default as it gave best cosine similarity scores as discovered later on, to be used throughout the experimentation

In [22]:
# Query example, used throughout the experimentation
question = "What are the main rules of football or soccer and what is a goal-kick and a corner-kick?"

In [23]:
# Retrieve the top-k documents that are most relevant to the query
top_k_indices, top_k_scores = retrieve_top_k_documents(question, documents, document_embeddings, k=3, search_type='cosine',fusion_k=60)

# Output the top 3 documents and their cosine similarity scores
print("Top 3 relevant documents based on cosine similarity:")
for idx, score in zip(top_k_indices, top_k_scores):
    print(f"Document: {documents[idx]} \nCosine Similarity Score: {score:.4f}\n")

# Retrieve top-k documents using BM25 keyword-based search
top_k_indices, top_k_scores = retrieve_top_k_documents(question, documents, document_embeddings, k=3, search_type='bm25',fusion_k=60)

# Output the top 3 documents based on BM25 keyword search
print("Top 3 relevant documents based on BM25 keyword search:")
for idx, score in zip(top_k_indices, top_k_scores):
    print(f"Document: {documents[idx]} \nKeyword Match Score: {score:.4f}\n")

# Retrieve top-k documents using hybrid search
top_k_indices, top_k_scores = retrieve_top_k_documents(question, documents, document_embeddings, k=3, search_type='hybrid',fusion_k=60)

# Output the top 3 documents based on hybrid search
print("Top 3 relevant documents based on hybrid search:")
for i, idx in enumerate(top_k_indices):
    print(f"[{i+1}] Score: {top_k_scores[i]:.4f}\nDoc: {documents[idx]}\n")

Top 3 relevant documents based on cosine similarity:
Document: Association football, more commonly known as football or soccer, is a team sport played between two teams of 11 players who almost exclusively use their feet to propel a ball around a rectangular field called a pitch. The objective of the game is to score more goals than the opposing team by moving the ball beyond the goal line into a rectangular-framed goal defended by the opposing team. Traditionally, the game has been played over two 45-minute halves, for a total match time of 90 minutes. With an estimated 250 million players active in over 200 countries and territories, it is the world's most popular sport. Association football is played in accordance with the Laws of the Game, a set of rules that has been in effect since 1863 and maintained by the IFAB since 1886. The game is played with a football that is 68â70 cm (27â28 in) in circumference. The two teams compete to score goals by getting the ball into the other 

Comparing the results of the three different retrival approaches, cosine, BM25 keyword, hybrid, limited to top 3 documents for ease of use

In [11]:
# Authenticate with Hugging Face (needed for gated models), used for testing bigger models, ended up not needed since the models we used didnt require authentication or access tokens
login()

In [12]:
#Zephyr 7B model for text generation

model_name = "HuggingFaceH4/zephyr-7b-beta"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto", #device_map='cuda'
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Loading the Zephyr 7B model, to later be used for output generation

In [13]:
# Create a pipeline
generator = pipeline(
"text-generation",
model=model,
tokenizer=tokenizer,
return_full_text=False,
max_new_tokens=256,
do_sample=False
)

Device set to use cuda:0


Defining the pipeline for the Zephyr 7B model to perform text generation

In [17]:
# Retrieve top-k documents using hybrid search
top_k_indices, top_k_scores = retrieve_top_k_documents(question, documents, document_embeddings, k=10, search_type='hybrid',fusion_k=60)

# Extract the top-k document contents into an array
retrieved_documents = [documents[i] for i in top_k_indices]

Document retrieval for a single query

In [15]:
#For better output, using multiple similar queries

# Query example
question = "What are the main rules of football or soccer and what is a goal-kick and a corner-kick?"

# Original query prompts
query_prompts = [
    "Can you explain the fundamental rules of football or soccer, including goal-kicks and corner-kicks?",
    "What are the basic rules in soccer, and could you also describe the differences between a goal-kick and a corner-kick?",
    "Tell me about the main regulations in football, especially focusing on goal-kicks and corner-kicks.",
    "What is the significance of goal-kicks and corner-kicks in football, and what are the primary rules of the game?",
    "How do the rules of soccer work, and can you explain when and why goal-kicks and corner-kicks are taken?"
]

# Store the generated queries as qs1, qs2, qs3, qs4, qs5
qs1, qs2, qs3, qs4, qs5 = query_prompts

retrieved_documents = []
retrieved_scores = []
document_rankings = defaultdict(list)  # To store rank information for RRF

# Loop through each query and retrieve top-k documents
for query_index, query in enumerate(query_prompts):
    top_k_indices, top_k_scores = retrieve_top_k_documents(query, documents, document_embeddings, k=5, search_type='hybrid', fusion_k=60)
    
    # Record the ranks and scores for RRF
    for rank, (doc_index, score) in enumerate(zip(top_k_indices, top_k_scores)):
        document_rankings[doc_index].append((query_index, rank + 1, score))  # Store query index, rank, and score

# Calculate Reciprocal Rank Fusion scores
rrf_scores = {}

# RRF Calculation
k_q = 60  # Constant
for doc_index, rankings in document_rankings.items():
    rrf_score = 0
    for query_index, rank, score in rankings:
        rrf_score += 1 / (k_q + rank)
    rrf_scores[doc_index] = rrf_score

# Sort documents by RRF score and select top-10
sorted_documents = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)[:10]

# Retrieve and print the top 10 documents
top_10_documents = [documents[doc_index] for doc_index, _ in sorted_documents]
top_10_scores = [rrf_scores[doc_index] for doc_index, _ in sorted_documents]

# Output the top 10 documents and their scores
for idx, (doc, score) in enumerate(zip(top_10_documents, top_10_scores)):
    print(f"Document {idx + 1} (Score: {score}):\n{doc}\n")

Document 1 (Score: 0.03225806451612903):
Association football, more commonly known as football or soccer, is a team sport played between two teams of 11 players who almost exclusively use their feet to propel a ball around a rectangular field called a pitch. The objective of the game is to score more goals than the opposing team by moving the ball beyond the goal line into a rectangular-framed goal defended by the opposing team. Traditionally, the game has been played over two 45-minute halves, for a total match time of 90 minutes. With an estimated 250 million players active in over 200 countries and territories, it is the world's most popular sport. Association football is played in accordance with the Laws of the Game, a set of rules that has been in effect since 1863 and maintained by the IFAB since 1886. The game is played with a football that is 68â70 cm (27â28 in) in circumference. The two teams compete to score goals by getting the ball into the other team's goal (between t

Document retrieval for 5 prompt engineered queries (optional) which resemble the original query, retrieves 5 documents per query (original + 5 generated) and uses RRF to discover the top 10 retrieved queries

In [20]:
# Load a summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Summarize each retrieved document
summarized_responses = []
for doc in top_10_documents: # use retrieved_documents if you want to avoid using the generated prompts, top_10_documents for generated prompts
    try:
        summary = summarizer(doc, max_length=250, min_length=100, do_sample=False)[0]["summary_text"]
        summarized_responses.append(summary)
    except Exception as e:
        print(f"Error summarizing document: {e}")
        summarized_responses.append(doc)  # fallback to original doc if summarization fails
        
# Print the summarized responses
for i, summary in enumerate(summarized_responses):
    print(f"Summary {i + 1}: {summary}\n")

Device set to use cuda:0


Summary 1:  association football is a team sport played between two teams of 11 players. The objective of the game is to score more goals than the opposing team. With an estimated 250 million players active in over 200 countries and territories, it is the world's most popular sport. The game is played with a football that is 68â70 cm (27â28 in) in circumference. The two teams compete to score goals by getting the ball into the other team's goal (between the posts, under the bar, and fully across the goal line)

Summary 2: In 2019, goals scored by hand, whether accidental or not, are disallowed. Attacking players can no longer interfere in defensive walls during free kicks. Substituted players have to leave the field at the nearest goal line or touchline instead of walking to their technical area. During penalties, goalkeepers are only required to keep one foot on the line. In 2023, "Kicks from the penalty mark" renamed to "Penalties (penalty shoot-out)". Cautions in regulation play

Summarization function (optional), uses Facebook Bart to summarize the retrieved documents to remove noise from documents before they are passed on to the text generation model.

In [19]:
# Construct the RAG prompt for a single query input and its resulting output
prompt = f"""
You are an AI assistant tasked with answering questions based on retrieved knowledge.

### **Retrieved Information**:
1. {summarized_responses[0]}

2. {summarized_responses[1]}

3. {summarized_responses[2]}

4. {summarized_responses[3]}

5. {summarized_responses[4]}

6. {summarized_responses[5]}

7. {summarized_responses[6]}

8. {summarized_responses[7]}

9. {summarized_responses[8]}

10. {summarized_responses[9]}

### **Question**:
{question}

### **Instructions**:
- Integrate the key points from all retrieved responses into a **cohesive, well-structured answer**.
- If the responses are **contradictory**, mention the different perspectives.
- If none of the retrieved responses contain relevant information, reply:
  **"I couldn't find a good response to your query in the database."**
"""

# Generate response using Zephyr-7B
messages = [{"role": "user", "content": prompt}]
output = generator(messages)
print(textwrap.fill(output[0]["generated_text"], width=80))

Football, also known as soccer or association football, is a team sport played
between two teams of 11 players. The objective is to score more goals than the
opposing team by getting the ball into their goal, which is located between the
posts and under the crossbar. The game is played with a spherical ball that is
68-70 cm in circumference, and it is prohibited for all players except the
goalkeeper to handle the ball with their hands, except during a throw-in.  In
addition to association football, there are other codes of football that share
similarities, such as rugby football, which split into rugby union, rugby
league, American football, and Canadian football. These codes allow carrying or
handling the ball by all players, unlike association football.  In association
football, there are specific rules for restarting play after the ball goes out
of bounds. A goal-kick is awarded to the defending team if the ball goes out of
play over their goal line. The goalkeeper takes the kick fr

RAG prompt for a single query as an input, to compare with that generated by multiple query inputs down below

In [21]:
#Rag prompt function which instead takes all the generated queries as inputs aswell as their relevant documents

# Function to generate the RAG prompt and get the response
def generate_rag_response(queries, summarized_responses, generator):
    # Initialize the prompt
    prompt = f"""
    You are an AI assistant tasked with answering questions based on retrieved knowledge.

    ### **Retrieved Information**:
    1. {summarized_responses[0]}
    2. {summarized_responses[1]}
    3. {summarized_responses[2]}
    4. {summarized_responses[3]}
    5. {summarized_responses[4]}
    6. {summarized_responses[5]}
    7. {summarized_responses[6]}
    8. {summarized_responses[7]}
    9. {summarized_responses[8]}
    10. {summarized_responses[9]}

    ### **Question**:
    {queries[0]}  # Original query

    ### **Generated Queries**:
    1. {queries[1]}  # Generated query 1
    2. {queries[2]}  # Generated query 2
    3. {queries[3]}  # Generated query 3
    4. {queries[4]}  # Generated query 4
    5. {queries[5]}  # Generated query 5

    ### **Instructions**:
    - Integrate the key points from all retrieved responses into a **cohesive, well-structured answer**.
    - If the responses are **contradictory**, mention the different perspectives.
    - If none of the retrieved responses contain relevant information, reply:
      **"I couldn't find a good response to your query in the database."**
    """

    # Generate response using the RAG model (Zephyr-7B)
    messages = [{"role": "user", "content": prompt}]
    output = generator(messages)
    
    # Print the generated response, wrapped for readability
    return textwrap.fill(output[0]["generated_text"], width=80)

# Example queries and summarized responses
queries = [question, qs1, qs2, qs3, qs4, qs5]  # Including original and generated queries

# Get the RAG response from the generator
rag_response = generate_rag_response(queries, summarized_responses, generator)

# Print the final generated answer
print(rag_response)


Certainly! Football, also known as soccer, is a team sport played between two
teams of eleven players. The objective is to score more goals than the opposing
team by getting the ball into their goal, which is located between the posts and
under the bar, and fully across the goal line. The game is played with a ball
that is 68-70 cm in circumference.  In football, there are two types of kicks
that are important to understand: goal-kicks and corner-kicks. A goal-kick is
awarded to the defending team if the ball goes out of play over their goal line,
provided that no player from the opposing team touched the ball last. The goal-
keeper or any other player on the defending team takes the goal-kick from any
point within the goal area and must kick the ball into play. The kick cannot be
played directly into the goal, and opponents must be at least 10 yards away
until the ball is in play.  A corner-kick, on the other hand, is awarded to the
attacking team if the ball goes out of play over the

RAG prompt function that takes all 6 queries as inputs and their RRF calculated documents to generate a hopefully better output